# 学習データを増やす

In [42]:
import os
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

def draw_images(generator, x, dir_name, index):
    # 出力ファイルの設定
    save_name = 'extened-' + str(index)
    g = generator.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix=save_name, save_format='jpg')

    # 1つの入力画像から何枚拡張するかを指定
    # g.next()の回数分拡張される
    for i in range(200):
        bach = g.next()


if __name__ == '__main__':

    # 拡張する際の設定
    generator = ImageDataGenerator(
                    rotation_range=90, # 90°まで回転
                    width_shift_range=0.0, # 水平方向にランダムでシフト
                    height_shift_range=0.0, # 垂直方向にランダムでシフト
                    channel_shift_range=0.0, # 色調をランダム変更
                    shear_range=0.00, # 斜め方向(pi/8まで)に引っ張る
                    horizontal_flip=True, # 垂直方向にランダムで反転
                    vertical_flip=True, # 水平方向にランダムで反転
                    zoom_range=0.2
                    )

    # りんごテストデータ
    # 出力先ディレクトリの設定
    output_dir = "data/test/exapple"
    if not(os.path.exists(output_dir)):
        os.mkdir(output_dir)

    # 拡張する画像群の読み込み
    images = glob.glob(os.path.join('data/test/apple/', "*.jpeg"))

    # 読み込んだ画像を順に拡張
    for i in range(len(images)):
        img = load_img(images[i])
        # 画像を配列化して転置a
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        # 画像の拡張
        draw_images(generator, x, output_dir, i)
        
    # オレンジテストデータ
    # 出力先ディレクトリの設定
    output_dir = "data/test/exorange"
    if not(os.path.exists(output_dir)):
        os.mkdir(output_dir)

    # 拡張する画像群の読み込み
    images = glob.glob(os.path.join('data/test/orange/', "*.jpeg"))

    # 読み込んだ画像を順に拡張
    for i in range(len(images)):
        img = load_img(images[i])
        # 画像を配列化して転置a
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        # 画像の拡張
        draw_images(generator, x, output_dir, i)
        

    # りんご訓練データ
    # 出力先ディレクトリの設定
    output_dir = "data/train/exapple"
    if not(os.path.exists(output_dir)):
        os.mkdir(output_dir)

    # 拡張する画像群の読み込み
    images = glob.glob(os.path.join('data/train/apple/', "*.jpeg"))

    # 読み込んだ画像を順に拡張
    for i in range(len(images)):
        img = load_img(images[i])
        # 画像を配列化して転置a
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        # 画像の拡張
        draw_images(generator, x, output_dir, i)

    # オレンジ訓練データ
    # 出力先ディレクトリの設定
    output_dir = "data/train/exorange"
    if not(os.path.exists(output_dir)):
        os.mkdir(output_dir)

    # 拡張する画像群の読み込み
    images = glob.glob(os.path.join('data/train/orange/', "*.jpeg"))

    # 読み込んだ画像を順に拡張
    for i in range(len(images)):
        img = load_img(images[i])
        # 画像を配列化して転置a
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        # 画像の拡張
        draw_images(generator, x, output_dir, i)

In [43]:
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adagrad
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

# 学習用のデータを作る.
image_list = []
label_list = []

# ./data/train 以下のorange,appleディレクトリ以下の画像を読み込む。
for dir in os.listdir("data/train"):
    if dir == ".DS_Store":
        continue

    dir1 = "data/train/" + dir 
    label = 0

    if (dir == "apple"): # appleはラベル0
        label = 0
    elif (dir == "exapple"):
        label = 0
    elif (dir == "orange"): # orangeはラベル1
        label = 1
    elif (dir == "exorange"):
        label = 1
    else:
        continue

    for file in os.listdir(dir1):
        if file != ".DS_Store":
            # 配列label_listに正解ラベルを追加(りんご:0 オレンジ:1)
            label_list.append(label)
            filepath = dir1 + "/" + file
            image = np.array(Image.open(filepath).resize((25, 25)))
            image_list.append(image / 255.)

# kerasに渡すためにnumpy配列に変換。
image_list = np.array(image_list)

# ラベルの配列を1と0からなるラベル配列に変更
# 0 -> [1,0], 1 -> [0,1] という感じ。
Y = to_categorical(label_list)

# モデルを作成
model = Sequential()
# model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(25, 25, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

## 学習のためのモデルを設定
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# 学習を実行。10%はテストに使用。
model.fit(image_list, Y, nb_epoch=20, batch_size=100, validation_split=0.1)

# 学習結果
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:98: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 4290 samples, validate on 477 samples
Epoch 1/20
4290/4290 [==============================] - 9s 2ms/step - loss: 0.2387 - acc: 0.9072 - val_loss: 0.0057 - val_acc: 1.0000
Epoch 2/20
4290/4290 [==============================] - 8s 2ms/step - loss: 0.0035 - acc: 1.0000 - val_loss: 5.6604e-04 - val_acc: 1.0000
Epoch 3/20
4290/4290 [==============================] - 8s 2ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.1265e-04 - val_acc: 1.0000
Epoch 4/20
4290/4290 [==============================] - 8s 2ms/step - loss: 5.2397e-04 - acc: 1.0000 - val_loss: 1.4486e-04 - val_acc: 1.0000
Epoch 5/20
4290/4290 [==============================] - 8s 2ms/step - loss: 2.5675e-04 - acc: 1.0000 - val_loss: 2.3502e-05 - val_acc: 1.0000
Epoch 6/20
4290/4290 [==============================] - 8s 2ms/step - loss: 1.8409e-04 - acc: 1.0000 - val_loss: 1.3195e-05 - val_acc: 1.0000
Epoch 7/20
4290/4290 [==============================] - 8s 2ms/step - loss: 1.0681e-04 - acc: 1.0000 - val_loss: 8.99

In [44]:
# テスト用ディレクトリ(./data/train/)の画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

for dir in os.listdir("data/train"):
    if dir == ".DS_Store":
        continue

    dir1 = "data/test/" + dir 
    label = 0

    if (dir == "apple"): # appleはラベル0
        label = 0
    elif (dir == "exapple"):
        label = 0
    elif (dir == "orange"): # orangeはラベル1
        label = 1
    elif (dir == "exorange"):
        label = 1
    else:
        continue

    for file in os.listdir(dir1):
        if file != ".DS_Store":
            label_list.append(label)
            filepath = dir1 + "/" + file
            image = np.array(Image.open(filepath).resize((25, 25)))
            print(filepath)
#             image = image.transpose(2, 0, 1)
#             image = image.reshape(1, image.shape[0] * image.shape[1] * image.shape[2]).astype("float32")[0]
            result = model.predict_classes(np.array([image / 255.]))
            print("label:", label, "result:", result[0])

            total += 1.

            if label == result[0]:
                ok_count += 1.

print("seikai: ", ok_count / total * 100, "%")

data/test/apple/21.jpeg
label: 0 result: 0
data/test/apple/22.jpeg
label: 0 result: 0
data/test/apple/23.jpeg
label: 0 result: 0
data/test/apple/24.jpeg
label: 0 result: 0
data/test/apple/25.jpeg
label: 0 result: 0
data/test/exapple/extened-0_0_870.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_1777.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_3923.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_5145.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_1463.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_9331.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_2929.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_7851.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_2366.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_7676.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_9541.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_8412.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_9101.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_82

data/test/exapple/extened-3_0_5624.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_7530.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_6213.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_3826.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_3355.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_5042.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_5056.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_9791.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_4360.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_685.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_8795.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_6021.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_7459.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_8599.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_7866.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_9906.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_5614.jpg
label: 0 result: 0
data/test/exapp

data/test/exapple/extened-3_0_979.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_2800.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_583.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_3459.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_5194.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_5230.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_5483.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_1342.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_4787.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_2360.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_5216.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_6674.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_458.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_7619.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_2995.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_829.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_7275.jpg
label: 0 result: 0
data/test/exapple/

label: 0 result: 0
data/test/exapple/extened-2_0_2349.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_2047.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_5164.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_947.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_4508.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_8082.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_1683.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_2539.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_3241.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_9161.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_3394.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_1980.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_8280.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_2347.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_2981.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_5849.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_9516.jpg
label: 0 result

label: 0 result: 0
data/test/exapple/extened-3_0_5275.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_5904.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_7548.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_3823.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_855.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_2283.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_2702.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_9825.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_3706.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_4874.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_4508.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_5643.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_3495.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_4647.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_4561.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_9723.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_802.jpg
label: 0 result:

label: 0 result: 0
data/test/exapple/extened-4_0_2535.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_9384.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_2652.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_4663.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_9353.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_3497.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_2951.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_2890.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_4211.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_8725.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_3454.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_4385.jpg
label: 0 result: 0
data/test/exapple/extened-3_0_5598.jpg
label: 0 result: 0
data/test/exapple/extened-0_0_7168.jpg
label: 0 result: 0
data/test/exapple/extened-2_0_5866.jpg
label: 0 result: 0
data/test/exapple/extened-4_0_2119.jpg
label: 0 result: 0
data/test/exapple/extened-1_0_766.jpg
label: 0 result

data/test/exorange/extened-2_0_5339.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_6399.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_4864.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_8152.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_8389.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_4871.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_5849.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_607.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_8344.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_1348.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_3461.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_9444.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_8982.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_9336.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_8766.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_948.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_983.jpg
label: 1 result: 1


label: 1 result: 1
data/test/exorange/extened-1_0_5626.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_1865.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_8597.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_8318.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_5628.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_8481.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_860.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_8100.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_6380.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_9356.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_5104.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_4568.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_7188.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_29.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_4867.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_5020.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_5746.jpg


data/test/exorange/extened-1_0_8379.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_2228.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_8808.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_9733.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_1822.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_9069.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_1279.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_5779.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_1523.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_7642.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_3356.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_8966.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_3166.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_581.jpg
label: 1 result: 1
data/test/exorange/extened-0_0_2162.jpg
label: 1 result: 1
data/test/exorange/extened-2_0_7332.jpg
label: 1 result: 1
data/test/exorange/extened-1_0_7102.jpg
label: 1 result: 

In [45]:
# 保存 ディレクトリに出力される
model.save("./model.h5")